In [1]:
import pandas as pd
import copy
import pickle
from itertools import cycle
from bertopic import BERTopic

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/

In [2]:
df_old = pd.read_csv("../data/csv_files/schneider.csv")

/tmp/ipykernel_8504/456796116.py:1: DtypeWarning: Columns (5,6,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_old = pd.read_csv("../data/csv_files/schneider.csv")


In [3]:
import sys
sys.path.append('../')

from utils.schneider import countries_to_update, text_data_column, words_to_filter, replacements
from preprocessing.dataLoaders.schneider_data_loader import SchneiderDataLoader
from preprocessing.preprocessing import Preprocessor

df = pd.read_csv("../data/csv_files/schneider_2023.csv")
schneiderDataLoader = SchneiderDataLoader(df, countries_to_update)
preprocessing = Preprocessor(
    schneiderDataLoader, 
    text_data_column,
    words_to_filter,
    replacements
)

df_2023_preprocessed_full = preprocessing.preprocess()
# Filter rows based on 'non_empty_rows' column
df_2023_preprocessed = df_2023_preprocessed_full[df_2023_preprocessed_full['non_empty_rows']]

In [21]:
def load_bertopic_model(filename):
    """
    Load a BERTopic model and associated data from a file.
    
    :param filename: The name of the file to load the data from.
    :return: A tuple containing the loaded BERTopic model, topics, probs, and docs variables.
    """
    # Load the BERTopic model
    topic_model = BERTopic.load(filename)
    
    # Load the topics, probs, and docs variables
    with open(filename + '_data.pkl', 'rb') as f:
        topics, probs, embeddings, docs = pickle.load(f)
    
    return topic_model, topics, probs, embeddings, docs

topic_model, topics, probs, embeddings, docs = load_bertopic_model('../models/raw_keybert_bertopic_model')

def create_merged_model(docs, bertopic_model, topics_to_merge_dict, label_names_dict):
    """
    Create a new BERTopic model by merging topics from an existing model.

    This function takes as input a list of documents `docs`, an existing BERTopic model `bertopic_model`, a dictionary `topics_to_merge_dict` specifying which topics to merge, and a dictionary `label_names_dict` specifying the labels for the merged topics.

    The function creates a deep copy of the input BERTopic model and merges the specified topics using the `merge_topics` method. Then, it sets the topic labels for the merged model using the `set_topic_labels` method and the provided `label_names_dict`.

    The resulting merged BERTopic model is then returned.

    Parameters:
        docs (list): A list of documents used to fit the BERTopic model.
        bertopic_model (BERTopic): The input BERTopic model to be merged.
        topics_to_merge_dict (dict): A dictionary specifying which topics to merge. The keys are the topic numbers to be merged, and the values are the topic numbers into which they should be merged.
        label_names_dict (dict): A dictionary specifying the labels for the merged topics. The keys are the topic numbers, and the values are the corresponding labels.

    Returns:
        BERTopic: The resulting merged BERTopic model.
    """
    topic_model_merged = copy.deepcopy(bertopic_model)
    topic_model_merged.merge_topics(docs, topics_to_merge_dict)

    # Create a dictionary to match the aggregated name to their corresponding topic number
    mergedtopic_labels_dict = {i-1: item for i, item in enumerate(label_names_dict)}
    # Set topic labels for the aggregated model
    topic_model_merged.set_topic_labels(mergedtopic_labels_dict)

    return topic_model_merged### Création du modèle bertopic aggrégé pour topics finaux

# List of topics numbers. Each value of this list is a list that contains the topic number of the topics to join together
topics_to_merge = [ [42,3,0,13], #Delivery Deadlines : challenges and strategies involved in managing delivery deadlines in logistics operations. (vert)
                    [20,50,27], #Quotation and Pricing Strategies (vert bas)
                    [35,32], #Touch Panels and Screens (rouge, haut)
                    [40,36], #Frequency Converters : frequency converters used in industrial applications and the technical support provided by manufacturers and suppliers (rouge, suite)
                    [37,21,6,12,9,4,1,14,16,31,19], #“Automation Components” : hardware and software components used in industrial automation systems. (rouge centre)
                    [33,46,8], #Product Evaluation : evaluate the quality, affordability and reliability of products and services (rouge, fin)
                    [44,51,23,41,49,57,22], #Customer Support : Reliability and Quality in Customer Service and Support (bleu ciel)
                    [58,59], #Quick Customer Service (marron)
                    [38,10,26,52,39,43], #Problem Solving and Communication (focus on the importance of being efficient and precise when solving problems) (jaune)
                    [45,47,55,53,54], #Assistance and Guidance (noir)
                    [29,30,11,24], #Power Supply Issues (2e vert, haut)
                    [7,5,2,25,15,34,18,28,17], #Technical Support (2e vert, bas)
                    [48,56] #None : positive feedback (2e rouge)
]

label_names_postmerge = [
    "Outliers",
    "Automation Components",
    "Technical Support",
    "Delivery Deadlines",
    "Problem Solving & Comm",
    "Power Supply Issues",
    "Customer Support", #Reliability and Quality in Customer Service and Support
    "Product Evaluation",
    "Pricing", #Quotation and Pricing Strategies
    "Assistance", #Assistance and Guidance
    "Touch Screens", #Touch Panels and Screens
    "Frequency Converters",
    "Positive feedback",
    "Quick Customer Service"
    ]

# Set the topic names for the new aggregated topic
# It must match the order from the topics_to_merge list
label_names = ["Delivery Deadlines",
    "Pricing", #Quotation and Pricing Strategies
    "Touch Screens", #Touch Panels and Screens
    "Frequency Converters",
    "Automation Components",
    "Product Evaluation",
    "Customer Support", #Reliability and Quality in Customer Service and Support
    "Quick Customer Service",
    "Problem Solving & Comm",
    "Assistance", #Assistance and Guidance
    "Power Supply Issues",
    "Technical Support",
    "positive feedback"]

topic_labels_dict = {}
topic_labels_dict[-1]="Outliers"
for i in range(len(topics_to_merge)):
    for topic in topics_to_merge[i]:
        topic_labels_dict[topic] = label_names[i]

# Create a new merged bertopic model 
topic_model_merged = create_merged_model(docs, topic_model, topics_to_merge, label_names_postmerge)

# topic_model_merged = copy.deepcopy(topic_model)
# topic_model_merged.set_topic_labels(topic_labels_dict)
# topic_model_merged.merge_topics(docs, topics_to_merge)

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


In [23]:
df_2023_preprocessed.reset_index(drop=True, inplace=True)
new_topics, new_probs = topic_model_merged.transform(df_2023_preprocessed['processed_data'])

# Now, you can merge the new dataframe with the old one
# df_labelled = pd.concat([df_labelled, df_2023_preprocessed])


In [24]:
# Get the topic and keywords for the new data
new_topic_keywords = {}
n = 10  # Number of keywords to include
for topic in topic_model_merged.get_topics().keys():
    topic_words = [word for word, _ in topic_model_merged.get_topic(topic)[:n]]
    new_topic_keywords[topic] = ", ".join(topic_words)

# Add the topics and keywords to the new dataframe
df_2023_preprocessed['topic'] = new_topics
df_2023_preprocessed['keywords'] = df_2023_preprocessed['topic'].map(new_topic_keywords)

# Add the label column to the new dataframe
mergedtopic_labels_dict = {i-1: item for i, item in enumerate(label_names_postmerge)}
df_2023_preprocessed['label'] = df_2023_preprocessed['topic'].map(mergedtopic_labels_dict)

# Get a list of the topics in the correct order, excluding topic -1
topic_order = [topic for topic in topic_model_merged.get_topics().keys() if topic != -1]

# Create a dictionary where the keys are the topic labels and the values are the probabilities
new_proba_dict = []
for proba in new_probs:
    new_proba_dict.append(dict(sorted(zip(topic_order, proba), key=lambda item: item[1], reverse=True)))
    
# Add the proba_dict column to the new dataframe
df_2023_preprocessed['proba_dict'] = new_proba_dict

In [25]:
df_2023_preprocessed.columns
# len(df_2023_preprocessed_full)
df_2023_preprocessed[["id","processed_data","topic","keywords","label","proba_dict"]].head(10)

,id,processed_data,topic,keywords,label,proba_dict
0,7,The case cannot be closed yet. The variator is...,-1,"customer service, delivery time, sales, techni...",Outliers,"{5: 0.4971376656629934, 1: 0.18351327926820915..."
1,12,You kindly provided additional information in ...,-1,"customer service, delivery time, sales, techni...",Outliers,"{3: 0.5921793282262936, 8: 0.02130923580118823..."
2,28,Delivery date adjustment not carried out!,2,"delivery time, delays, delivery date, deliveri...",Delivery Deadlines,"{2: 0.37335230490884497, 0: 0.1247352399500280..."
3,29,"Very good service, very interactive and the at...",1,"technical support, response time, customer ser...",Technical Support,"{1: 0.9511487162523243, 5: 3.0232526310585277e..."
4,36,Reliable and quick offer,7,"pricing, prices, discount, quotation, orders, ...",Pricing,"{7: 0.054763016492342505, 0: 0.039560743994227..."
5,40,Nothing. for now we are satisfied with both th...,7,"pricing, prices, discount, quotation, orders, ...",Pricing,"{7: 0.6985103184838195, 0: 0.00366780982522584..."
6,42,Even though 2 months have passed since the ord...,-1,"customer service, delivery time, sales, techni...",Outliers,"{5: 0.15332537003148491, 1: 0.0869908212703196..."
7,43,Nice nd excellent work by Schneider worker spa...,0,"technical support, programmable logic controll...",Automation Components,"{0: 0.2672348851927576, 5: 0.04677892258495698..."
8,45,"Mr. Melih Solmaz followed the issue closely, a...",-1,"customer service, delivery time, sales, techni...",Outliers,"{5: 0.15871219627779123, 1: 0.0970748022941245..."
9,46,I received help with concise and quick answers...,-1,"customer service, delivery time, sales, techni...",Outliers,"{1: 0.21914335128108423, 5: 0.1672696371400092..."
